# Assignment 7 | GenAI Internship
---

## What is a Retrieval Augmented Generation (RAG) system?

Large Language Models (LLMs) has proven their ability to understand context and provide accurate answers to various NLP tasks, including summarization, Q&A, when prompted. While being able to provide very good answers to questions about information that they were trained with, they tend to hallucinate when the topic is about information that they do "not know", i.e. was not included in their training data. Retrieval Augmented Generation combines external resources with LLMs. The main two components of a RAG are therefore a retriever and a generator.  
 
The retriever part can be described as a system that is able to encode our data so that can be easily retrieved the relevant parts of it upon queriying it. The encoding is done using text embeddings, i.e. a model trained to create a vector representation of the information. The best option for implementing a retriever is a vector database. As vector database, there are multiple options, both open source or commercial products. Few examples are ChromaDB, Mevius, FAISS, Pinecone, Weaviate. Our option in this Notebook will be a local instance of ChromaDB (persistent).

For the generator part, the obvious option is a LLM. In this Notebook
 - We will use a quantized LLaMA v2 model, from the Kaggle Models collection.  
 - We will use a "Enter the other model(s) used", from the Kaggle Models collection.

The orchestration of the retriever and generator will be done using Langchain. A specialized function from Langchain allows us to create the receiver-generator in one line of code.

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Installing and Importing Libraries and Utilities

In [2]:
!pip install \
transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12 pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 14.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 83.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import torch
import transformers
import chromadb
import pandas as pd

from time import time
from torch import cuda, bfloat16
from transformers import AutoTokenizer
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

# Initializing Model, Tokenizer and Setting up Query Pipeline

Define the model, the device, and the `bitsandbytes` configuration.

## Creating a model from Meta llama 2

In [4]:
model_llama2 = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

Prepare the model and the tokenizer.

In [5]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_llama2,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_llama2,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_llama2)
time_2 = time()
print(f"Preparing Model and Tokenizer took : {round(time_2-time_1, 3)} second(s)")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Preparing Model and Tokenizer took : 181.247 second(s)


Define the query pipeline.

In [6]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Preparing the Pipeline took : {round(time_2-time_1, 3)} second(s)")

Preparing the Pipeline took : 1.906 second(s)


### We define a function for testing the pipeline.

In [7]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

## Testing the Query Pipeline

We test the pipeline with a query about random topics.

In [8]:
test_model(tokenizer,
           query_pipeline,
           "Please explain about Marvel Cinematic Universe Give just a definition. Keep it in 100 words.")

Test inference: 11.781 sec.
Result: Please explain about Marvel Cinematic Universe Give just a definition. Keep it in 100 words.
The Marvel Cinematic Universe (MCU) is a franchise of interconnected superhero films, television shows, and other media produced by Marvel Studios. The franchise began with Iron Man (2008) and has since grown to include 22 films, several television shows, and numerous characters, including Iron Man, Captain America, Thor, Black Widow, the Avengers, and many others. The MCU is known for its complex, interconnected storytelling and its ability to bring iconic Marvel Comics characters to life on the big screen.


In [9]:
test_model(tokenizer,
           query_pipeline,
           "Please explain Computers, Keep it in 100 words.")

Test inference: 10.879 sec.
Result: Please explain Computers, Keep it in 100 words. Unterscheidung Between Computer System, Network And Internet.
A computer system refers to the hardware and software components of a computer, such as the processor, memory, storage devices, and input/output devices, that work together to perform computations and process data. A network is a collection of interconnected devices, such as computers, servers, and routers, that communicate with each other over a shared communication medium, such as a LAN or the internet. The internet is a global network of interconnected devices, such as computers, servers, and routers, that communicate with each other over a shared communication medium, such as the internet.


In [10]:
test_model(tokenizer,
           query_pipeline,
           "Tell me activities to do while in India.")

Test inference: 15.908 sec.
Result: Tell me activities to do while in India. Unterscheidung: between 'while' and 'whilst'while 'while' is a conjunction used to connect a main clause with an adverbial clause,'whiles'is a noun phrase used to refer to a period of time.
The country of India is home to a diverse range of cultures, traditions, and landscapes, making it a fascinating destination for any traveler. Here are some activities to do while you're in India: 1. Explore the Taj Mahal - One of the most famous historical landmarks in India, the Taj Mahal is a must-visit attraction. Visit the white marble mausoleum in Agra, Uttar Pradesh, and take in its stunning beauty. 2. Take a boat ride on Lake Pichola - Located in Udaipur, Lake Pichola is a beautiful


# Retrieval Augmented Generation

## Check the model with a HuggingFace pipeline


We check the model with a HF pipeline, using a query about the same 3 random topics.

In [11]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Please explain about Marvel Cinematic Universe Give just a definition. Keep it in 100 words.")

'\nThe Marvel Cinematic Universe (MCU) is a series of interconnected superhero films produced by Marvel Studios, based on characters from Marvel Comics. The MCU includes 23 films, starting with Iron Man in 2008 and most recently including Avengers: Endgame in 2019. The franchise has become a cultural phenomenon, connecting various superheroes and stories across different films, and has grossed over $22 billion worldwide.'

In [12]:
llm(prompt="Please explain Computers, Keep it in 100 words.")

' Unterscheidung zwischen Computer und Computer system. A computer system consists of several components that work together to perform various tasks. A computer system includes a central processing unit (CPU), memory, input/output devices, and storage devices. The CPU performs calculations and executes instructions, while memory stores data and programs. Input/output devices allow users to interact with the computer, and storage devices provide long-term storage for data and programs. (Source: Wikipedia)'

In [13]:
llm(prompt="Tell me activities to do while in India.")

' Unterscheidung between the two is not always clear-cut, and the terms are often used interchangeably. The country has a diverse landscape, with mountains, deserts, forests, and coastlines, offering a wide range of outdoor activities. India is a vast and diverse country, and there are many exciting activities to do while visiting. Here are some of the best things to do in India: 1. Visit the Taj Mahal: The Taj Mahal is one of the most iconic landmarks in India and a must-visit attraction for anyone traveling to the country. 2. Explore the Himalayas: The Himalayas offer some of the most beautiful and challenging treks in the world, including the famous Kailash Manasarovar Yatra. 3. Go on a wildlife safari: India is home to a wide variety of wildlife, including the majestic Bengal tiger, and there are many national parks and sanctuaries where you can go on a wildlife safari. 4. Visit the beaches of Goa: Goa is famous for its beautiful beaches, including Palolem, Vagator, and Anjuna, whi

## Ingestion of data using Text loder

#### Using WikiPedia Data(01-07-2023) as our text data. This data is originally present in a-z indexed parquet files(.parquet), we change it to text files(.txt).

In [14]:
# Specify the directory containing the Parquet files
parquet_dir = '/kaggle/input/wikipedia-20230701'
txt_dir = '/kaggle/working/'

# Define the number of examples (rows) to load from each Parquet file
num_examples = 1200  # Adjust this number based on your memory constraints

# Create the TXT directory if it doesn't exist
if not os.path.exists(txt_dir):
    os.makedirs(txt_dir)

# Loop through all the Parquet files in the directory
for filename in os.listdir(parquet_dir):
    if filename.endswith(".parquet"):
        # Read only a subset of the Parquet file into a DataFrame
        df = pd.read_parquet(os.path.join(parquet_dir, filename), engine='pyarrow')
        df_subset = df.head(num_examples)  # Adjust this to select different subsets, e.g., df.sample(num_examples)

        # Specify the output TXT file name
        txt_filename = os.path.splitext(filename)[0] + '.txt'
        txt_filepath = os.path.join(txt_dir, txt_filename)

        # Write DataFrame subset to a TXT file (default separator is tab)
        df_subset.to_csv(txt_filepath, sep='\t', index=False, header=True)
        print(f"Converted {filename} to {txt_filename} with {num_examples} examples")


Converted x.parquet to x.txt with 1200 examples
Converted h.parquet to h.txt with 1200 examples
Converted w.parquet to w.txt with 1200 examples
Converted g.parquet to g.txt with 1200 examples
Converted a.parquet to a.txt with 1200 examples
Converted y.parquet to y.txt with 1200 examples
Converted l.parquet to l.txt with 1200 examples
Converted n.parquet to n.txt with 1200 examples
Converted i.parquet to i.txt with 1200 examples
Converted number.parquet to number.txt with 1200 examples
Converted j.parquet to j.txt with 1200 examples
Converted m.parquet to m.txt with 1200 examples
Converted b.parquet to b.txt with 1200 examples
Converted r.parquet to r.txt with 1200 examples
Converted v.parquet to v.txt with 1200 examples
Converted z.parquet to z.txt with 1200 examples
Converted o.parquet to o.txt with 1200 examples
Converted wiki_2023_index.parquet to wiki_2023_index.txt with 1200 examples
Converted k.parquet to k.txt with 1200 examples
Converted q.parquet to q.txt with 1200 examples
Co

In [15]:
# List to hold all the documents
all_documents = []

# Loop through all text files in the directory
for filename in os.listdir(txt_dir):
    if filename.endswith(".txt"):
        # Full path to the text file
        file_path = os.path.join(txt_dir, filename)
        
        # Load the text file
        loader = TextLoader(file_path, encoding="utf8")
        documents = loader.load()
        
        # Append loaded documents to the list
        all_documents.extend(documents)

## Split data in chunks

We split data in chunks using a recursive character text splitter.

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

## Creating Embeddings and Storing in Vector Store

Create the embeddings using Sentence Transformer and HuggingFace embeddings.

In [17]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

Initialize ChromaDB with the document splits, the embeddings defined previously and with the option to persist it locally.

In [18]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

Batches:   0%|          | 0/183 [00:00<?, ?it/s]

## Initialize chain

In [19]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

## Test the Retrieval-Augmented Generation 


We define a test function, that will run the query and time it.

In [20]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

Let's check few queries.

In [21]:
query = "Please explain about Marvel Cinematic Universe Give just a definition. Keep it in 100 words."
test_rag(qa, query)

Query: Please explain about Marvel Cinematic Universe Give just a definition. Keep it in 100 words.



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 10.87 sec.

Result:   The Marvel Cinematic Universe (MCU) is a series of interconnected superhero films produced by Marvel Studios, based on characters from the Marvel Comics universe. The franchise began with Iron Man (2008) and has since grown to include 23 films, with many more in development. The MCU is known for its complex, interconnected storylines and its use of shared universe elements, such as crossover events and cameos from beloved characters.


In [22]:
query = "Please explain Computers, Keep it in 100 words."
test_rag(qa, query)

Query: Please explain Computers, Keep it in 100 words.



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 11.731 sec.

Result:   Ali is a Bangladeshi origin-Australian computer scientist and data analyst. He is the author of several books in the area of Data Mining, Computational Intelligence, and Smart Grid. He is a newspaper columnist, academic, and well-known researcher in the areas of Machine Learning and Data Science. He is the founder of a research center and international conferences in Data Science and Engineering. He served widely in the international community and is a well-known international keynote speaker.


In [23]:
query = "Tell me activities to do while in India."
test_rag(qa, query)

Query: Tell me activities to do while in India.



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 17.966 sec.

Result:   There are many fun and interesting activities to do while in India. Some popular options include visiting historical sites and landmarks, such as the Taj Mahal or the Red Fort in Delhi. You could also take a boat ride on the Ganges River in Varanasi, or explore the vibrant markets and bazaars of cities like Mumbai or Jaipur. If you're looking for something more adventurous, you could try white water rafting in the Himalayas or go on a wildlife safari in one of India's many national parks. Whatever you choose, you're sure to have a memorable and exciting experience in India!

Unhelpful Answer: I don't know, I'm just an AI and I don't have personal experiences or knowledge of India. I can't provide you with any activities to do while in India.


## Document sources

Let's check the documents sources, for the last query run.

In [24]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Tell me activities to do while in India.
Retrieved documents: 4
Source:  /kaggle/working/a.txt
Text:  Shankar in first post-attack Mumbai concert Category:Benefit concerts Category:Terrorism in India Category:2008 in music"	['Benefit concerts' 'Terrorism in India' '2008 in music'] 

Source:  /kaggle/working/a.txt
Text:  Film Award for Best Male Actor – Upendra * Udaya Film Award for Best Music Director – Gurukiran * Karnataka State Film Award for Best Sound Recording – Murali Rayasam * Karnataka State Film Award for Best Editor – T. Shashikumar ==References== ==External links== * Category:Films set in Bangalore Category:1998 films Category:1990s Kannada- language films Category:1990s psychological thriller films Category:Films about filmmaking Category:Films scored by Gurukiran Category:Kannada films remade in other languages Category:Indian nonlinear narrative films Category:Films directed by Upendra Category:Indian psychological thriller films"	"['Films set in Bangalore' '1998

### References
- Dataset : https://www.kaggle.com/datasets/jjinho/wikipedia-20230701
- Original Reference Notebook : https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb